# Supervised Classification: Decision Trees, SVM, and Naive Bayes | Assignment Solution

This notebook provides the complete solution for the Supervised Classification Assignment (DA-AG-009).

## Question 1: What is Information Gain, and how is it used in Decision Trees?

**Information Gain (IG)** is a metric used by Decision Tree algorithms (like ID3 and C4.5) to decide which feature to split on at each node in order to build the tree.

* **Definition:** It measures the reduction in **Entropy** (or impurity) of a dataset after splitting it based on an attribute. In essence, it tells us how much 'useful' information a particular feature provides.
* **Usage in Decision Trees:** At every step, the Decision Tree algorithm calculates the Information Gain for every remaining feature. It selects the feature that yields the **highest Information Gain** to make the split. This ensures that the tree partitions the data most effectively, moving toward purer child nodes and reducing the overall tree depth. The goal is to maximize the purity of the resulting subsets.

## Question 2: What is the difference between Gini Impurity and Entropy?

Both **Gini Impurity** and **Entropy** are metrics used by Decision Trees to measure the impurity or randomness of a node, helping the algorithm decide the best split.

| Feature | Gini Impurity (Used by CART) | Entropy (Used by ID3, C4.5) |
| :--- | :--- | :--- |
| **Definition** | Measures the probability of incorrectly classifying a randomly chosen element in the dataset if it were randomly labeled according to the class distribution. | Measures the expected amount of information (or randomness) required to classify a data point in the set. |
| **Mathematical Form** | $\sum_{i=1}^{C} p_i (1 - p_i)$ | $-\sum_{i=1}^{C} p_i \log_2(p_i)$ |
| **Range** | 0 (pure) to 0.5 (maximum impurity, for 2 classes) | 0 (pure) to 1 (maximum impurity, for 2 classes) |
| **Computation** | Involves squaring probabilities, making it computationally faster. | Involves logarithmic calculations, which can be slightly slower. |

**Key Difference:** While they often lead to similar tree structures, Gini tends to isolate the most frequent class in its own branch, while Entropy tends to produce slightly more balanced trees. However, due to its simplicity and computational efficiency, **Gini Impurity is often the default choice** in many modern libraries like scikit-learn.

## Question 3: What is Pre-Pruning in Decision Trees?

**Pre-pruning** is a technique used to **prevent overfitting** in Decision Tree models by stopping the tree construction process *before* it reaches full depth.

* **Mechanism (The "Stop Early" Approach):** The algorithm applies constraints during the training phase. If a node does not meet a specified criterion, the splitting process for that branch is halted, and the node is made a leaf node.
* **Common Criteria:**
    1.  **Maximum Depth (`max_depth`):** Limiting the maximum number of levels in the tree.
    2.  **Minimum Samples in a Leaf (`min_samples_leaf`):** Requiring a minimum number of samples in any terminal node.
    3.  **Minimum Impurity Decrease (`min_impurity_decrease`):** Requiring a split to provide a minimum reduction in impurity (Gini or Entropy).

Pre-pruning generally results in a smaller, simpler tree, which helps **reduce variance and improve generalization** to unseen data.

## Question 4: Write a Python program to train a Decision Tree Classifier using Gini Impurity as the criterion and print the feature importances (practical).

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# 1. Generate synthetic data (4 features, 2 informative)
X, y = make_classification(
    n_samples=1000,
    n_features=4,
    n_informative=2,
    n_redundant=0,
    random_state=42
)
feature_names = ['Feature A', 'Feature B', 'Feature C', 'Feature D']

# 2. Split data (optional but good practice)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initialize and train the Decision Tree Classifier using Gini
dt_classifier = DecisionTreeClassifier(criterion='gini', random_state=42)
dt_classifier.fit(X_train, y_train)

# 4. Print feature importances
print("--- Decision Tree Feature Importances (Gini) ---")
importances = dt_classifier.feature_importances_
for name, importance in sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True):
    print(f"{name}: {importance:.4f}")

# Note: Features A and C should show higher importance as they were set as informative
from sklearn.metrics import accuracy_score
y_pred = dt_classifier.predict(X_test)
print(f"\nTest Accuracy: {accuracy_score(y_test, y_pred):.4f}")

--- Decision Tree Feature Importances (Gini) ---
Feature C: 0.6713
Feature A: 0.2215
Feature D: 0.0575
Feature B: 0.0497

Test Accuracy: 0.8900


## Question 5: What is a Support Vector Machine (SVM)?

A **Support Vector Machine (SVM)** is a powerful and versatile supervised machine learning algorithm used for both classification and regression, though it's primarily known for classification.

* **Goal:** The main objective of an SVM is to find the optimal **hyperplane** (a decision boundary) that distinctly classifies the data points. This hyperplane maximizes the **margin**—the distance between the hyperplane and the nearest data points of any class.
* **Support Vectors:** The data points closest to the hyperplane are called **Support Vectors**. These points are crucial because they alone define the position and orientation of the hyperplane, making the model robust even with large datasets. 

## Question 6: What is the Kernel Trick in SVM?

The **Kernel Trick** is a fundamental technique that allows SVMs to perform effective classification even when the data is **not linearly separable** in its original feature space.

* **Problem:** If data points cannot be separated by a straight line (or flat hyperplane), standard linear SVM fails.
* **Solution:** The Kernel Trick uses a **kernel function** (e.g., Polynomial, Radial Basis Function/RBF) to implicitly map the data points from the low-dimensional feature space to a **much higher-dimensional feature space** where the data *can* be linearly separated. Crucially, the kernel function does this mapping without ever explicitly calculating the coordinates in the new, higher-dimensional space. It computes the dot product of the mapped points directly in the original space, saving huge amounts of computational cost.
* **Benefit:** It allows us to fit non-linear models while keeping the computational simplicity of a linear algorithm.

## Question 7: Write a Python program to train two SVM classifiers with Linear and RBF kernels on the Wine dataset, then compare their accuracies.

In [3]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# 1. Load and prepare data
wine = load_wine()
X, y = wine.data, wine.target

# Scale features (important for SVM, especially RBF kernel)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# 2. Train Linear Kernel SVM
svm_linear = SVC(kernel='linear', random_state=42)
svm_linear.fit(X_train, y_train)
acc_linear = accuracy_score(y_test, svm_linear.predict(X_test))

# 3. Train RBF Kernel (Non-linear) SVM
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, y_train)
acc_rbf = accuracy_score(y_test, svm_rbf.predict(X_test))

# 4. Compare Accuracies
print("--- SVM Kernel Comparison (Wine Dataset) ---")
print(f"Accuracy (Linear Kernel): {acc_linear:.4f}")
print(f"Accuracy (RBF Kernel): {acc_rbf:.4f}")

if acc_rbf > acc_linear:
    print("\nObservation: The RBF (non-linear) kernel provided a slightly better accuracy, suggesting the data is not perfectly linearly separable.")
elif acc_linear > acc_rbf:
    print("\nObservation: The Linear kernel performed better, suggesting the data is either linear or the RBF hyper-parameters need tuning.")
else:
    print("\nObservation: Both kernels performed equally well.")

--- SVM Kernel Comparison (Wine Dataset) ---
Accuracy (Linear Kernel): 0.9815
Accuracy (RBF Kernel): 0.9815

Observation: Both kernels performed equally well.


## Question 8: What is the Naïve Bayes classifier, and why is it called "Naïve"?

The **Naïve Bayes classifier** is a family of probabilistic classification algorithms based on **Bayes' Theorem**.

* **Bayes' Theorem:** It calculates the probability of a hypothesis (a data point belonging to a certain class) given the evidence (the features of the data point).

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

* **Why it's "Naïve":** The classifier is called *Naïve* because it makes the fundamental, simplifying, and often unrealistic assumption of **conditional independence** among the features. 

    * In simple terms, it assumes that the presence of a specific feature in a class is unrelated to the presence of any other feature. For example, in a text classification problem, Naïve Bayes assumes that the probability of the word "yellow" appearing is independent of the word "taxi" appearing, given the category (e.g., transportation). Despite this strong and sometimes false assumption, Naïve Bayes often performs surprisingly well in practice, especially in text processing.

## Question 9: Explain the differences between Gaussian Naïve Bayes, Multinomial Naïve Bayes, and Bernoulli Naïve Bayes.

The three primary variants of Naïve Bayes classifiers differ based on the type of data they are designed to model and the underlying statistical distribution they assume for the features ($P(B|A)$ in Bayes' Theorem).

| Variant | Feature Type | Typical Application |
| :--- | :--- | :--- |
| **Gaussian Naïve Bayes** | **Continuous** (e.g., Height, Weight, Sensor Readings) | Assumes that continuous features associated with each class are distributed according to a **Gaussian (Normal) distribution**. |
| **Multinomial Naïve Bayes** | **Count Data/Discrete** (e.g., Word counts in a document) | Used when features represent the frequency of events (counts), most commonly for **text classification** where features are word counts or frequencies. |
| **Bernoulli Naïve Bayes** | **Binary/Boolean** (e.g., Feature present or absent, 0 or 1) | Used when features are binary. In text classification, it checks for the **presence or absence** of a word in a document, rather than its frequency. |

The key is matching the model type to the data's distribution.

## Question 10: Write a Python program to train a Gaussian Naïve Bayes classifier on the Breast Cancer dataset and evaluate accuracy.

In [5]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 1. Load the dataset
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# 2. Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 3. Initialize and train the Gaussian Naïve Bayes classifier
# GaussianNB is used because the features are continuous measurements
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# 4. Make predictions and evaluate accuracy
y_pred = gnb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("--- Gaussian Naïve Bayes (Breast Cancer Dataset) ---")
print(f"Model Accuracy: {accuracy:.4f}")
print("\nInterpretation: A high accuracy indicates that the Naïve Bayes model, despite its independence assumption, is effective at classifying benign vs. malignant tumors based on the continuous feature measurements (like mean radius, texture, etc.) in this dataset.")

--- Gaussian Naïve Bayes (Breast Cancer Dataset) ---
Model Accuracy: 0.9415

Interpretation: A high accuracy indicates that the Naïve Bayes model, despite its independence assumption, is effective at classifying benign vs. malignant tumors based on the continuous feature measurements (like mean radius, texture, etc.) in this dataset.
